In [1]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

#lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
lista=['FLMUSDT', 'DODOXUSDT', 'XRPUSDT', 'MATICUSDT', 'SUIUSDT', 'BELUSDT', 'TRBUSDT', 'PEOPLEUSDT', 'ETHUSDT', 'NMRUSDT', 'BNBUSDT', 'PERPUSDT', 'LINKUSDT', 'BCHUSDT', 'DOGEUSDT', '1000PEPEUSDT', 'SOLUSDT', 'LTCUSDT', 'ENJUSDT', 'SNXUSDT', 'BLZUSDT', 'CYBERUSDT', 'CFXUSDT', 'BTCUSDT', 'LPTUSDT', 'WLDUSDT', 'UNFIUSDT']
#lista=['LPTUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data,porcentajeentrada = md.estrategia_atrapes(symbol,tp_flag = False, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo,porcentajeentrada=porcentajeentrada)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


FLMUSDT - Return [%]: 0.08% - # Trades: 4 - Profit Factor: 3.395241382276425 - Win Rate [%]: 25.0
DODOXUSDT - Return [%]: -0.24% - # Trades: 5 - Profit Factor: 0.13449342864165695 - Win Rate [%]: 40.0
XRPUSDT - Return [%]: 0.15% - # Trades: 4 - Profit Factor: 3.4859377590922014 - Win Rate [%]: 75.0
MATICUSDT - Return [%]: -0.14% - # Trades: 4 - Profit Factor: 0.467962705510432 - Win Rate [%]: 25.0
SUIUSDT - Return [%]: 0.12% - # Trades: 6 - Profit Factor: 1.3321187087522823 - Win Rate [%]: 33.33333333333333
BELUSDT - Return [%]: 0.33% - # Trades: 6 - Profit Factor: 1.237487948663451 - Win Rate [%]: 33.33333333333333
TRBUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
PEOPLEUSDT - Return [%]: 0.0% - # Trades: 4 - Profit Factor: 0.3011103714074663 - Win Rate [%]: 25.0
ETHUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
NMRUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
BNBUSDT - Return [%]: 0.0% - 